In [1]:
from argparse import ArgumentParser
import pytorch_lightning as pl
from pl_bolts.datamodules.cifar10_datamodule import CIFAR10DataModule
from pl_bolts.datamodules.stl10_datamodule import STL10DataModule
from pl_bolts.models.self_supervised import SimCLR, CPC_v2, SwAV
from pl_bolts.models.self_supervised.resnets import resnet50
from torchsummary import summary
from tabulate import tabulate


import torch
import torch.nn as nn
import numpy
import os
import datetime
from src.feature_tsne import featureToTSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [5]:
batch_size = 512
CIFAR10DataModule().prepare_data()
dm = CIFAR10DataModule(data_dir="data", batch_size=batch_size, num_workers=8)
dm.prepare_data()
dm.setup()
classNames = ["airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
device = torch.device('cuda:0')
model_encoder = resnet50(first_conv= False, maxpool1=False)
fv_size = 32768
weight_path = '/home/hjsong/EmbeddingAnalysis/logs/1208/cifar10/100epochs/adam/version_3/checkpoints/epoch=95-step=16895.ckpt'
model = SimCLR.load_from_checkpoint(weight_path, strict=False)
model_encoder.load_state_dict(model.encoder.state_dict(), strict=False)
model_encoder.avgpool = nn.Identity()
model_encoder.fc = nn.Identity()        
model_encoder.to(device)
model_encoder.eval()
train_data_encoded = torch.zeros([len(dm.dataset_train), fv_size]).to(device)

with torch.no_grad():
    for xi, x in enumerate(dm.train_dataloader()):
        input, label = x
        encoded = model_encoder(input.to(device))
        train_data_encoded[xi*batch_size:min((xi+1)*batch_size, len(dm.dataset_train))] = encoded[0]
        
features = train_data_encoded.cpu().detach().numpy()
